In [1]:
!nvidia-smi

Mon Sep 21 12:29:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

170500096/170498071 [==============================] - 8s 0us/step


In [4]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
print(train_images.shape)
print(train_labels.shape)

(50000, 32, 32, 3)
(50000, 1)


In [6]:
train_labels[0]

array([6], dtype=uint8)

In [8]:
# 스케일링만
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

In [10]:
from tensorflow.keras.utils import to_categorical

In [11]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [13]:
train_labels[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [14]:
from tensorflow.keras import models, layers

In [36]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=10, activation='softmax'))

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)               

In [38]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [45]:
history = model.fit(x=train_images, y=train_labels,
                    epochs=20,
                    batch_size=64,
                    validation_split=0.2)

Epoch 1/20
625/625 [==============================] - 3s 4ms/step - loss: 2.3027 - accuracy: 0.1000 - val_loss: 2.3027 - val_accuracy: 0.0997
Epoch 2/20
625/625 [==============================] - 3s 4ms/step - loss: 2.3027 - accuracy: 0.0991 - val_loss: 2.3027 - val_accuracy: 0.0952
Epoch 3/20
625/625 [==============================] - 3s 4ms/step - loss: 2.3027 - accuracy: 0.1001 - val_loss: 2.3028 - val_accuracy: 0.0952
Epoch 4/20
625/625 [==============================] - 3s 4ms/step - loss: 2.3027 - accuracy: 0.0989 - val_loss: 2.3027 - val_accuracy: 0.0952
Epoch 5/20
625/625 [==============================] - 3s 4ms/step - loss: 2.3027 - accuracy: 0.0977 - val_loss: 2.3027 - val_accuracy: 0.0952
Epoch 6/20
625/625 [==============================] - 3s 4ms/step - loss: 2.3027 - accuracy: 0.0984 - val_loss: 2.3027 - val_accuracy: 0.0952
Epoch 7/20
625/625 [==============================] - 3s 4ms/step - loss: 2.3027 - accuracy: 0.0998 - val_loss: 2.3027 - val_accuracy: 0.0952
Epoch 

In [24]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 2.3510 - accuracy: 0.1048


In [34]:
from tensorflow import keras

In [41]:
callbacks = [
             keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3),
             keras.callbacks.ModelCheckpoint('./checkpoint',
                                             monitor='val_accuracy', save_best_only=True)
]

In [42]:
callback_model = models.Sequential()
callback_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
callback_model.add(layers.MaxPooling2D((2, 2)))
callback_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
callback_model.add(layers.MaxPooling2D((2, 2)))
callback_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

callback_model.add(layers.Flatten())
callback_model.add(layers.Dense(units=64, activation='relu'))
callback_model.add(layers.Dense(units=10, activation='softmax'))

In [43]:
callback_model.compile(optimizer='rmsprop',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

In [44]:
history = callback_model.fit(x=train_images, y=train_labels,
                             epochs=30,
                             batch_size=64,
                             validation_split=0.2,
                             callbacks=callbacks)

Epoch 1/30
614/625 [============================>.] - ETA: 0s - loss: 2.2191 - accuracy: 0.1586WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./checkpoint/assets
625/625 [==============================] - 3s 5ms/step - loss: 2.2171 - accuracy: 0.1596 - val_loss: 2.0911 - val_accuracy: 0.2348
Epoch 2/30
625/625 [==============================] - 3s 5ms/step - loss: 2.0661 - accuracy: 0.2454 - val_loss: 2.0317 - val_accuracy: 0.2671
Epoch 3/30
625/625 [==============================] - 3s 5ms/step - loss: 1.9629 - accuracy: 0.2867 - val

In [46]:
from tensorflow.keras import models

In [48]:
load_model = models.load_model('./checkpoint')

In [50]:
test_loss, test_acc = load_model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 508.1149 - accuracy: 0.1710
